In [ ]:
import numpy as np
import json
import pandas as pd
import copy
import datetime
import matplotlib.pyplot as plt
import seaborn
import timeit
import pickle
import time
from pyfmi import load_fmu
import scipy
import sys
import os


# User Inputs

In [ ]:
# User inputs can be entered here
weather_filename='weather_KABQ.csv'
price_filename='CFH_Prices.csv'
fmu_name='R32AirToWaterHeatPumpWithTank'
#fmu_name='R32AirToWaterHeatPumpWithThermalStorage'


In [ ]:
start_time=10000
final_time=11800
step_size = 600
data_interval_length=60

# System Configuration 

In [ ]:
fmu_path=f'FmuModels/{fmu_name}/{fmu_name}.fmu'
fmu_data_points_path=f"FmuModels/{fmu_name}/{fmu_name}_data_points.csv"

In [ ]:
sys.path.append(os.path.join(sys.path[0],'FmuModels',fmu_name))

In [ ]:
exec(f'from {fmu_name}_Controller import MyController')
my_controller=MyController()


# Display a list of all data points from the FMU

In [ ]:
dataPointsFmu = pd.read_csv(fmu_data_points_path)
dataPointsFmu=dataPointsFmu[["points","keys","writable_point",'initial_value','descriptions']]
dataPointsFmu

# Load Weather Data

In [ ]:
df_weather = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'resources', 'weather', weather_filename), parse_dates = True)
df_weather

# Load Price Data

In [ ]:
df_price = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'resources', 'electricity_prices', price_filename), parse_dates = True)
df_price

# Load FMU

In [ ]:
model = load_fmu(fmu_path)

In [ ]:
#variable_values = dict(zip(dataPointsFmu['points'], dataPointsFmu['initial_value']))
#set(variable_values.keys())
#variable_values

# Create FMU simulation

In [ ]:
current_time=start_time
iteration=1
variable_values = dict(zip(dataPointsFmu['points'], dataPointsFmu['initial_value']))

while current_time+step_size <= final_time:
    
    global variable_values

    current_iteration_start_time=current_time
    current_iteration_final_time=current_time+step_size
    

    # call the controller
    current_inputs_dict=my_controller.controllerFunction(current_iteration_start_time, current_iteration_final_time, variable_values, df_weather, df_price)  


    
    
    
    
    # Sample list of names
    points_list = list(current_inputs_dict.keys())

    # Filter the DataFrame to select rows where "names" match the names in name_list
    filtered_dataPointsFmu = dataPointsFmu[dataPointsFmu['points'].isin(points_list)]

    # Extract the "keys" column from the filtered DataFrame and convert it to a Python list
    keys_list = filtered_dataPointsFmu['keys'].tolist()

    current_inputs_dict_processed = dict(zip(keys_list, list(current_inputs_dict.values())))
    current_inputs_dict_processed

    inputs = (
        list(current_inputs_dict_processed), 
        np.array(
            [[current_iteration_start_time]+list(current_inputs_dict_processed.values()),
             [current_iteration_final_time]+list(current_inputs_dict_processed.values())]
        )
    )    
    
    
    
    
    
    
    if iteration==1:
        opts = model.simulate_options()
        opts["ncp"] = (current_iteration_final_time-current_iteration_start_time)/data_interval_length
    else:
        state = model.get_fmu_state()
        opts['initialize'] = False
        opts["ncp"] = (current_iteration_final_time-current_iteration_start_time)/data_interval_length
        model.set_fmu_state(state)
        


    res = model.simulate(start_time=current_iteration_start_time, final_time=current_iteration_final_time,input=inputs, options=opts)
    
    !python FmuSimulationConfigFiles/MAT_to_CSV_conversion.py {fmu_name+'_result.mat'} {fmu_data_points_path}
    if iteration==1:
        df = pd.read_csv(fmu_name+'_result.csv')
    else:
        df_current = pd.read_csv(fmu_name+'_result.csv')
        df_current=df_current.tail(-1)
        df=pd.concat([df, df_current], ignore_index=True)
    
    
    variable_values=df.iloc[-1].drop(labels=['Unnamed: 0','time']).to_dict()

    
    iteration+=1
    current_time += step_size

In [ ]:

df=df.drop(['Unnamed: 0'], axis=1)


In [ ]:
try:
    os.remove(fmu_name+'_result.mat')
except:
    pass

try:
    os.remove(fmu_name+'_result.csv')
except:
    pass
try:
    os.remove(fmu_name+'_log.txt')
except:
    pass

# Simulation Result Table

In [ ]:
df

# Available data points for plotting

In [ ]:
df.dtypes

In [ ]:
"""
fig, (ax) = plt.subplots(nrows = 1, ncols = 1,figsize = (16,10))


ax.plot(df['time'],df['Tank Water Temperature Layer 1 [deg-K]'], label = 'Tank Water Temperature Layer 1 [deg-K]')
ax.plot(df['time'],df['Tank Water Temperature Layer 2 [deg-K]'], label = 'Tank Water Temperature Layer 2 [deg-K]')
ax.plot(df['time'],df['Tank Water Temperature Layer 3 [deg-K]'], label = 'Tank Water Temperature Layer 3 [deg-K]')
ax.plot(df['time'],df['Tank Water Temperature Layer 4 [deg-K]'], label = 'Tank Water Temperature Layer 4 [deg-K]')
ax.plot(df['time'],df['Tank Water Temperature Layer 5 [deg-K]'], label = 'Tank Water Temperature Layer 5 [deg-K]')


ax.set_xlabel('Time', fontsize = 17)
ax.set_ylabel('Temperature', fontsize = 17)
ax.legend(fontsize = 15)

ax.set_title('Temperature', fontsize = 18)
"""